In [1]:
import os

os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from dask.diagnostics import ProgressBar
from pathlib import Path
import libs.utils
import libs.vars
import numpy as np
import xarray

xarray.set_options(keep_attrs=True);

In [3]:
# ----- SPECIFIC SETTINGS -----
component = 'Amon'

files = [
    f'{{variable_id}}_{component}_{{source_id}}_historical_{{variant_label}}_{{grid_label}}_198001-201412_processed.nc',
    f'{{variable_id}}_{component}_{{source_id}}_ssp585_{{variant_label}}_{{grid_label}}_201501-210012_processed.nc'
]

In [4]:
def create_evspsbl(ensemble, files):
    for i, item in enumerate(ensemble):
        source_id = item['source_id']
        variant_label = item['variant_label']
        grid_label = 'gn'
        if source_id in ['EC-Earth3', 'IPSL-CM6A-LR']:
            grid_label = 'gr'
        
        format_vars = {
            'source_id': source_id,
            'variant_label': variant_label,
            'grid_label': grid_label
        }
        
        for filename in files:
            base_path = f'_data/cmip6/{source_id}/{{variable_id}}/'
            filepath_pr = (base_path + filename).format(variable_id='pr', **format_vars)
            filepath_evspsbl = (base_path + filename).format(variable_id='evspsbl', **format_vars)
            filepath_prnet = (base_path + filename).format(variable_id='prnet', **format_vars)
            
            if not Path(filepath_pr).exists():
                continue

            data_pr = xarray.open_mfdataset(paths=filepath_pr, combine='by_coords', use_cftime=True)
            data_evspsbl = xarray.open_mfdataset(paths=filepath_evspsbl, combine='by_coords', use_cftime=True)
            
            if source_id in ['EC-Earth3']:
                data_evspsbl *= -1
            
            # Calculate net pr (pr - evspsbl)
            data_pr['pr'] -= data_evspsbl['evspsbl']
            data_prnet = data_pr.rename({ 'pr': 'prnet' })
            
            Path(filepath_prnet).parent.mkdir(parents=True, exist_ok=True)
            print(filepath_prnet)
            
            if Path(filepath_prnet).exists():
                print('   -> Exists. skipping')
                continue

            write = data_prnet.to_netcdf(
                filepath_prnet,
                compute=False,
                engine='netcdf4',
                unlimited_dims=['time']
            )
            with ProgressBar():
                write.compute()

            data_pr.close()
            data_evspsbl.close()
            print('   -> Saved to disk')

            # Finally, compress as to_netcdf() seems to produce large file sizes
            file, diff = libs.utils.compress_nc_file(filepath_prnet, filepath_prnet)
            print(f'   -> Compressed ({diff})')

In [5]:
ensemble = libs.vars.ensemble()
create_evspsbl(ensemble, files)

_data/cmip6/ACCESS-CM2/prnet/prnet_Amon_ACCESS-CM2_historical_r1i1p1f1_gn_198001-201412_processed.nc
   -> Exists. skipping
_data/cmip6/ACCESS-CM2/prnet/prnet_Amon_ACCESS-CM2_ssp585_r1i1p1f1_gn_201501-210012_processed.nc
   -> Exists. skipping
_data/cmip6/CanESM5/prnet/prnet_Amon_CanESM5_historical_r1i1p2f1_gn_198001-201412_processed.nc
   -> Exists. skipping
_data/cmip6/CanESM5/prnet/prnet_Amon_CanESM5_ssp585_r1i1p2f1_gn_201501-210012_processed.nc
   -> Exists. skipping
_data/cmip6/EC-Earth3/prnet/prnet_Amon_EC-Earth3_historical_r4i1p1f1_gr_198001-201412_processed.nc
   -> Exists. skipping
_data/cmip6/EC-Earth3/prnet/prnet_Amon_EC-Earth3_ssp585_r4i1p1f1_gr_201501-210012_processed.nc
   -> Exists. skipping
_data/cmip6/HadGEM3-GC31-MM/prnet/prnet_Amon_HadGEM3-GC31-MM_historical_r1i1p1f3_gn_198001-201412_processed.nc
   -> Exists. skipping
_data/cmip6/HadGEM3-GC31-MM/prnet/prnet_Amon_HadGEM3-GC31-MM_ssp585_r1i1p1f3_gn_201501-210012_processed.nc
   -> Exists. skipping
_data/cmip6/IPSL-CM6